In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader 

from tqdm import tqdm
import heapq
import csv

import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import pandas as pd
import wandb


In [2]:
def data_load(path):
    # input - English
    # output - Urdu
    df = pd.read_csv(path,sep='\t', header=None)
    input_data = df[1].tolist()
    output_data = df[0].tolist()
    return input_data, output_data
    
def create_char_set(train, val):
    char_set = set()
    for word in train:
        for char in str(word):
            char_set.add(char)
    for word in val:
        for char in str(word):
            char_set.add(char)
    return char_set

def check_for_floats(data_list):
    float_values = []
    for item in data_list:
        if isinstance(item, float):
            float_values.append(item)
    return float_values

In [3]:
train_input1, train_output = data_load("/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/ur/lexicons/ur.translit.sampled.train.tsv")
val_input, val_output = data_load("/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/ur/lexicons/ur.translit.sampled.dev.tsv")
test_input, test_output = data_load("/kaggle/input/dakshina-dataset/dakshina_dataset_v1.0/ur/lexicons/ur.translit.sampled.test.tsv") 

print("Number of training samples: ", len(train_input1))
print("Number of validation samples: ", len(val_input))
print("Number of test samples: ", len(test_input))


Number of training samples:  106260
Number of validation samples:  10424
Number of test samples:  10517


In [4]:
eng_chars = create_char_set(train_input1, val_input)
print("Total English characters: ",len(eng_chars))
print(sorted(eng_chars))


Total English characters:  26
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
ur_chars = create_char_set(train_output, val_output)
print("Total Urdu characters: ",len(ur_chars))
print(sorted(ur_chars))


Total Urdu characters:  54
['ء', 'آ', 'ؤ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ي', 'ً', 'َ', 'ُ', 'ِ', 'ّ', 'ٗ', 'ٰ', 'ٹ', 'پ', 'چ', 'ڈ', 'ڑ', 'ژ', 'ک', 'گ', 'ں', 'ھ', 'ہ', 'ۃ', 'ی', 'ے', 'ۓ']


In [6]:
float_values=check_for_floats(train_input1)
print(float_values)

[nan, nan]


In [7]:
train_input = [x for x in train_input1 if not isinstance(x,float)]
print("Number of training samples: ", len(train_input))

Number of training samples:  106258


In [8]:
max_seq_eng = len(max(train_input+val_input+test_input, key=len))
max_seq_ur = len(max(train_output+val_output+test_output, key=len))
print("Length of the longest English word in corpus:",max_seq_eng)
print("Length of the longest Urdu word in corpus::",max_seq_ur)


Length of the longest English word in corpus: 21
Length of the longest Urdu word in corpus:: 14


In [11]:
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

def load_data(input_data, output_data, batch_size=32):
    """
    Prepares character-level transliteration data for sequence-to-sequence modeling.

    Args:
        input_data (list or Series): Romanized source words (e.g., Urdu in Latin script).
        output_data (list or Series): Target words in Devanagari/Urdu script.
        batch_size (int): Batch size for the DataLoader.

    Returns:
        dataset (Dataset): Custom PyTorch dataset for transliteration.
        dataloader (DataLoader): DataLoader for iterating over the dataset.
        input_vocab (dict): Character-to-index mapping for input characters.
        target_vocab (dict): Character-to-index mapping for target characters.
        max_input_len (int): Maximum input sequence length.
        max_target_len (int): Maximum target sequence length.
    """

    # Determine the maximum lengths for padding
    max_input_len = len(max(input_data, key=len))
    max_target_len = len(max(output_data, key=len))

    # Initialize character vocabularies with special tokens
    input_vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2}
    target_vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2}
    next_index = 3

    # Extract all unique characters from the dataset
    all_input_chars = ''.join(input_data)
    all_target_chars = ''.join(output_data)

    # Create input vocabulary
    for char in sorted(set(all_input_chars)):
        input_vocab[char] = next_index
        next_index += 1

    # Reset index for building target vocabulary
    next_index = 3
    for char in sorted(set(all_target_chars)):
        if char not in target_vocab:
            target_vocab[char] = next_index
            next_index += 1

    # Tokenize input characters and pad to max_input_len
    def tokenize_input(word, vocab, max_len):
        token_ids = [vocab[char] for char in word if char in vocab]
        padded = token_ids[:max_len] + [vocab['<pad>']] * (max_len - len(token_ids))
        return torch.tensor(padded)

    # Tokenize target characters with <sos> and <eos>, then pad
    def tokenize_target(word, vocab, max_len):
        token_ids = [vocab[char] for char in word if char in vocab]
        padded = [vocab['<sos>']] + token_ids[:max_len] + [vocab['<eos>']]
        padded += [vocab['<pad>']] * (max_len + 2 - len(padded))  # +2 for <sos> and <eos>
        return torch.tensor(padded)

    # Define a custom dataset for transliteration pairs
    class TransliterationDataset(Dataset):
        def __init__(self, input_words, target_words, input_vocab, target_vocab, max_input_len, max_target_len):
            self.input_words = input_words
            self.target_words = target_words
            self.input_vocab = input_vocab
            self.target_vocab = target_vocab
            self.max_input_len = max_input_len
            self.max_target_len = max_target_len

        def __len__(self):
            return len(self.input_words)

        def __getitem__(self, idx):
            input_word = self.input_words[idx]
            target_word = self.target_words[idx]

            # Convert to tensor of indices
            input_tensor = tokenize_input(input_word, self.input_vocab, self.max_input_len)
            target_tensor = tokenize_target(target_word, self.target_vocab, self.max_target_len)

            return input_tensor, target_tensor

    # Create dataset and dataloader
    dataset = TransliterationDataset(input_data, output_data,
                                     input_vocab, target_vocab,
                                     max_input_len, max_target_len)

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    return dataset, dataloader, input_vocab, target_vocab, max_input_len, max_target_len


In [12]:
dataset, dataloader, input_vocab, target_vocab, max_input_len, max_target_len = load_data(train_input+val_input+test_input, train_output+val_output+test_output,batch_size = 64)
print('All English Characters:\n',input_vocab,'\n All Urdu Characters:\n', target_vocab,'\n Length of  longest English word:', max_input_len,'\n Length of  longest Urdu word:', max_target_len) 

All English Characters:
 {'<pad>': 0, '<sos>': 1, '<eos>': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28} 
 All Urdu Characters:
 {'<pad>': 0, '<sos>': 1, '<eos>': 2, 'ء': 3, 'آ': 4, 'ؤ': 5, 'ئ': 6, 'ا': 7, 'ب': 8, 'ت': 9, 'ث': 10, 'ج': 11, 'ح': 12, 'خ': 13, 'د': 14, 'ذ': 15, 'ر': 16, 'ز': 17, 'س': 18, 'ش': 19, 'ص': 20, 'ض': 21, 'ط': 22, 'ظ': 23, 'ع': 24, 'غ': 25, 'ف': 26, 'ق': 27, 'ك': 28, 'ل': 29, 'م': 30, 'ن': 31, 'ه': 32, 'و': 33, 'ي': 34, 'ً': 35, 'َ': 36, 'ُ': 37, 'ِ': 38, 'ّ': 39, 'ٗ': 40, 'ٰ': 41, 'ٹ': 42, 'پ': 43, 'چ': 44, 'ڈ': 45, 'ڑ': 46, 'ژ': 47, 'ک': 48, 'گ': 49, 'ں': 50, 'ھ': 51, 'ہ': 52, 'ۃ': 53, 'ی': 54, 'ے': 55, 'ۓ': 56} 
 Length of  longest English word: 21 
 Length of  longest Urdu word: 14


## Seq2Seq Model

In [21]:
# Encoder: Encodes input sequence into context vector
class Encoder(nn.Module):
    def __init__(self, input_vocab_size, hidden_size, embedding_dim,
                 num_layers=1, dropout=0.5, cell_type='gru', bidirectional=False):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        self.bidirectional = bidirectional

        self.embedding = nn.Embedding(input_vocab_size, embedding_dim)
        self.dropout = nn.Dropout(dropout)

        rnn_cls = {'lstm': nn.LSTM, 'gru': nn.GRU, 'rnn': nn.RNN}[cell_type]
        self.rnn = rnn_cls(
            embedding_dim, hidden_size, num_layers,
            dropout=dropout, bidirectional=bidirectional, batch_first=True
        )

    def forward(self, input_seq):
        # Embed and apply dropout
        embedded = self.dropout(self.embedding(input_seq))

        # Pass through RNN
        rnn_output, hidden_state = self.rnn(embedded)

        # Handle LSTM: return both hidden and cell state
        if self.cell_type == 'lstm':
            hidden, cell = hidden_state
            if self.bidirectional:
                # Combine forward and backward hidden states by summing
                return torch.sum(hidden[-2:], dim=0, keepdim=True), torch.sum(cell[-2:], dim=0, keepdim=True)
            else:
                return hidden[-1].unsqueeze(0), cell[-1].unsqueeze(0)
        else:
            # GRU or RNN
            if self.bidirectional:
                return torch.sum(hidden_state[-2:], dim=0, keepdim=True)
            else:
                return hidden_state[-1].unsqueeze(0)


# Decoder: Generates output sequence using encoder context
class Decoder(nn.Module):
    def __init__(self, hidden_size, embedding_dim, output_vocab_size,
                 num_layers=1, dropout=0.5, cell_type='gru'):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type

        self.embedding = nn.Embedding(output_vocab_size, embedding_dim)
        self.dropout = nn.Dropout(dropout)

        rnn_cls = {'lstm': nn.LSTM, 'gru': nn.GRU, 'rnn': nn.RNN}[cell_type]
        self.rnn = rnn_cls(
            embedding_dim, hidden_size, num_layers,
            dropout=dropout, batch_first=True
        )

        self.fc = nn.Linear(hidden_size, output_vocab_size)

    def forward(self, input_token, hidden_state):
        # Add time dimension for RNN
        input_token = input_token.unsqueeze(1)

        # Embed input token and apply dropout
        embedded = self.dropout(self.embedding(input_token))

        # Pass through decoder RNN
        rnn_output, new_hidden_state = self.rnn(embedded, hidden_state)

        # Predict next token from output
        logits = self.fc(rnn_output)

        return logits.squeeze(1), new_hidden_state


# Sequence-to-Sequence: Combines encoder and decoder for full translation
class Seq2Seq(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size, hidden_size, embedding_dim,
                 encoder_layers=1, decoder_layers=1, dropout=0.3, cell_type='gru', bidirectional=True):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(input_vocab_size, hidden_size, embedding_dim,
                               encoder_layers, dropout, cell_type, bidirectional)
        self.decoder = Decoder(hidden_size, embedding_dim, output_vocab_size,
                               decoder_layers, dropout, cell_type)
        self.output_vocab_size = output_vocab_size

    def forward(self, input_seq, target_seq, teacher_forcing_ratio=0.5):
        batch_size = input_seq.size(0)
        target_seq_len = target_seq.size(1)

        outputs = torch.zeros(batch_size, target_seq_len, self.output_vocab_size).to(input_seq.device)

        # Encode the input sequence
        encoder_hidden = self.encoder(input_seq)

        # Initialize decoder hidden state from encoder
        decoder_hidden = self._init_decoder_hidden(encoder_hidden)

        # First decoder input is <sos>
        decoder_input = target_seq[:, 0]

        for t in range(1, target_seq_len):
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[:, t] = decoder_output

            # Teacher forcing: use actual target or model prediction
            use_teacher_forcing = torch.rand(1).item() < teacher_forcing_ratio
            next_input = target_seq[:, t] if use_teacher_forcing else decoder_output.argmax(1)
            decoder_input = next_input

        return outputs

    def _init_decoder_hidden(self, encoder_hidden):
        """Adjust encoder output to match decoder initial state shape."""
        decoder_layers = self.decoder.num_layers

        if self.decoder.cell_type == 'lstm':
            hidden, cell = encoder_hidden
            hidden_layers = hidden.shape[0]

            # Pad or trim hidden/cell states to match decoder's expected layers
            if hidden_layers < decoder_layers:
                pad_size = decoder_layers - hidden_layers
                hidden = torch.cat([hidden, torch.zeros(pad_size, *hidden.shape[1:], device=hidden.device)], dim=0)
                cell = torch.cat([cell, torch.zeros(pad_size, *cell.shape[1:], device=cell.device)], dim=0)
            else:
                hidden = hidden[:decoder_layers]
                cell = cell[:decoder_layers]
            return (hidden, cell)

        else:
            # GRU or RNN
            hidden_layers = encoder_hidden.shape[0]
            hidden = encoder_hidden
            if hidden_layers < decoder_layers:
                pad_size = decoder_layers - hidden_layers
                hidden = torch.cat([hidden, torch.zeros(pad_size, *hidden.shape[1:], device=hidden.device)], dim=0)
            else:
                hidden = hidden[:decoder_layers]
            return hidden


## Train and Evaluate

In [46]:
# Training function: Trains model for one epoch
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for latin_inputs, urdu_targets in dataloader:
        latin_inputs = latin_inputs.to(device)
        urdu_targets = urdu_targets.to(device)

        optimizer.zero_grad()
        output = model(latin_inputs, urdu_targets)
        output_dim = output.shape[-1]

        output_flat = output.view(-1, output_dim)
        targets_flat = urdu_targets.view(-1)

        loss = criterion(output_flat, targets_flat)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        # Compute accuracy (character-level)
        _, predictions = torch.max(output, dim=2)
        correct = (predictions == urdu_targets).all(dim=0).sum().item()
        total_correct += correct
        total_samples += urdu_targets.size(0)

    accuracy = 100* total_correct / total_samples
    return model, total_loss / len(dataloader), accuracy



# Evaluation function: Evaluates model performance on validation/test data
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch_latin, batch_urdu in dataloader:
            batch_latin = batch_latin.to(device)
            batch_urdu = batch_urdu.to(device)

            # Forward pass with no teacher forcing during evaluation
            predictions = model(batch_latin, batch_urdu, teacher_forcing_ratio=0.0)
            vocab_size = predictions.shape[-1]

            # Compute loss
            loss = criterion(predictions.view(-1, vocab_size), batch_urdu.view(-1))
            total_loss += loss.item()

            # Get predicted token indices
            _, predicted_indices = torch.max(predictions, dim=2)

            # Optional: Adjust predictions if vocab has special token offsets
            invalid_token_mask = predicted_indices > 9
            predicted_indices[invalid_token_mask] -= 2

            # Compare full sequences for exact word match
            correct_predictions = (predicted_indices == batch_urdu).all(dim=0).sum().item()

            total_correct += correct_predictions
            total_samples += batch_urdu.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = 100*(total_correct / total_samples)

    return avg_loss, accuracy



In [23]:

# Model configuration
input_vocab_size = 26         # Number of Latin script characters
output_vocab_size = 54        # Number of Urdu script characters
embedding_dim = 128           # Embedding dimension for both encoder and decoder
hidden_size = 128             # Hidden state size for RNN cells
encoder_layers = 3            # Number of layers in encoder RNN
decoder_layers = 2            # Number of layers in decoder RNN
cell_type = 'lstm'            # RNN cell type: 'rnn', 'gru', or 'lstm'
batch_size = 64               # Batch size during training
num_epochs = 20               # Total number of training epochs
dropout = 0.2                 # Dropout probability
learning_rate = 0.001         # Learning rate for optimizer
bidirectional = True          # Whether the encoder is bidirectional

# Initialize model, loss function, and optimizer
model = Seq2Seq(input_vocab_size, output_vocab_size, hidden_size, embedding_dim,
                encoder_layers, decoder_layers, dropout, cell_type, bidirectional)

print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(26, 128)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): LSTM(128, 128, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(54, 128)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): LSTM(128, 128, num_layers=2, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=128, out_features=54, bias=True)
  )
)


In [18]:
wandb.login(key='53b259076c07d0811d73bf26bfef7437e04dbf66')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [16]:
# Sweep configuration for hyperparameter optimization using Weights & Biases
sweep_config = {
    'method': 'bayes',  # Bayesian optimization for more efficient search
    'metric': {
        'name': 'val_accuracy',  # Metric to optimize
        'goal': 'maximize'       # Maximize validation accuracy
    },
    'parameters': {
        'embedding_size': {
            'values': [16,32, 64, 128, 256]  # Size of embedding vectors
        },
        'dropout': {
            'values': [0.0,0.2,0.4]  # Dropout probability to reduce overfitting
        },
        'encoder_layers': {
            'values': [1,2,3]  # Number of RNN layers in the encoder
        },
        'decoder_layers': {
            'values': [1,2,3]  # Number of RNN layers in the decoder
        },
        'hidden_size': {
            'values': [16,32, 64, 128, 256]  # Hidden state dimension in RNN
        },
        'rnn_cell_type': {
            'values': ['lstm','gru','rnn']  # RNN cell type to use
        },
        'use_bidirectional_encoder': {
            'values': [True, False]  # Whether encoder is bidirectional
        },
        'batch_size': {
            'values': [32, 64]  # Batch size during training
        },
        'num_epochs': {
            'values': [8,10]  # Number of training epochs
        },
        'learning_rate': {
            'values': [0.01, 0.001]  # Learning rate for optimizer
        }
    }
}

# Launch sweep on Weights & Biases
sweep_id = wandb.sweep(sweep=sweep_config, project='DA6401_Assignment-3')


Create sweep with ID: qr8bd7h5
Sweep URL: https://wandb.ai/ma23m013-iit-madras/DA6401_Assignment-3/sweeps/qr8bd7h5


In [17]:
def run_training():
    '''
    This function is executed by WandB for each set of hyperparameters during the sweep.
    It initializes the model using the current configuration, trains and evaluates it,
    and logs the metrics to Weights & Biases.
    '''
    with wandb.init() as run:
        # Create a descriptive run name using the current hyperparameters
        run_name = (
            f"cell-{wandb.config.rnn_cell_type}"
            f"_encLayers-{wandb.config.encoder_layers}"
            f"_decLayers-{wandb.config.decoder_layers}"
            f"_dropout-{wandb.config.dropout}"
            f"_embedSize-{wandb.config.embedding_size}"
            f"_hiddenSize-{wandb.config.hidden_size}"
            f"_batchSize-{wandb.config.batch_size}"
            f"_epochs-{wandb.config.num_epochs}"
            f"_lr-{wandb.config.learning_rate}"
        )
        wandb.run.name = run_name

        # Instantiate model
        model = Seq2Seq(
            input_vocab_size=30,
            output_vocab_size=60,
            hidden_size=wandb.config.hidden_size,
            embedding_dim=wandb.config.embedding_size,
            encoder_layers=wandb.config.encoder_layers,
            decoder_layers=wandb.config.decoder_layers,
            dropout=wandb.config.dropout,
            cell_type=wandb.config.rnn_cell_type,
            bidirectional=wandb.config.use_bidirectional_encoder
        )
        print(model)

        # Setup
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)

        # Load data
        _, train_loader, _, _, _, _ = load_data(train_input,train_output, batch_size=wandb.config.batch_size)

        _, val_loader, _, _, _, _ = load_data(val_input,val_output, batch_size=wandb.config.batch_size)

        # Train & evaluate
        for epoch in range(wandb.config.num_epochs):
            model, train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)
            val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)

            # Log metrics
            wandb.log({
                'Epoch': epoch,
                'Train Loss': train_loss,
                'Train Accuracy (%)': train_accuracy,
                'Validation Loss': val_loss,
                'Validation Accuracy (%)': val_accuracy
            })

            print(f"Epoch {epoch + 1}/{wandb.config.num_epochs} | "
                  f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}% | "
                  f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%")

# Start the sweep
wandb.agent(sweep_id, function=run_training, count=1)
wandb.finish()


wandb: Agent Starting Run: ha8idzhq with config:
wandb: 	batch_size: 64
wandb: 	decoder_layers: 2
wandb: 	dropout: 0
wandb: 	embedding_size: 64
wandb: 	encoder_layers: 2
wandb: 	hidden_size: 32
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 8
wandb: 	rnn_cell_type: rnn
wandb: 	use_bidirectional_encoder: True
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(30, 64)
    (dropout): Dropout(p=0, inplace=False)
    (rnn): RNN(64, 32, num_layers=2, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(60, 64)
    (dropout): Dropout(p=0, inplace=False)
    (rnn): RNN(64, 32, num_layers=2, batch_first=True)
    (fc): Linear(in_features=32, out_features=60, bias=True)
  )
)
Epoch 1/8 | Train Loss: 1.4623 | Train Accuracy: 11.84% | Val Loss: 2.0143 | Val Accuracy: 8.62%
Epoch 2/8 | Train Loss: 1.3423 | Train Accuracy: 12.10% | Val Loss: 2.0995 | Val Accuracy: 8.62%
Epoch 3/8 | Train Loss: 1.3032 | Train Accuracy: 12.10% | Val Loss: 1.9375 | Val Accuracy: 8.62%
Epoch 4/8 | Train Loss: 1.2791 | Train Accuracy: 12.03% | Val Loss: 1.9296 | Val Accuracy: 8.68%
Epoch 5/8 | Train Loss: 1.2613 | Train Accuracy: 12.01% | Val Loss: 1.9489 | Val Accuracy: 8.70%
Epoch 6/8 | Train Loss: 1.2536 | Train Accuracy: 12.06% | Val Loss: 1.9815 | Val Accuracy: 8.70%
Epoch 7/

Epoch,▁▂▃▄▅▆▇█
Train Accuracy (%),▁██▆▅▇▆▆
Train Loss,█▄▃▂▂▁▁▁
Validation Accuracy (%),▁▁▁▆▇▇▇█
Validation Loss,▄█▁▁▂▃▂▂
Epoch,7
Train Accuracy (%),12.01227
Train Loss,1.24084
Validation Accuracy (%),8.71067
Validation Loss,1.95907


# Test Accuracy for Best Model configuration

In [43]:
# Best Model configuration
input_vocab_size = 30         # Number of Latin script characters
output_vocab_size = 58        # Number of Urdu script characters
embedding_dim = 64           # Embedding dimension for both encoder and decoder
hidden_size = 128             # Hidden state size for RNN cells
encoder_layers = 3            # Number of layers in encoder RNN
decoder_layers = 3            # Number of layers in decoder RNN
cell_type = 'lstm'            # RNN cell type: 'rnn', 'gru', or 'lstm'
batch_size = 64               # Batch size during training
num_epochs = 10               # Total number of training epochs
dropout = 0.2                 # Dropout probability
learning_rate = 0.001         # Learning rate for optimizer
bidirectional = True          # Whether the encoder is bidirectional

# Initialize model, loss function, and optimizer
model = Seq2Seq(input_vocab_size, output_vocab_size, hidden_size, embedding_dim,
                encoder_layers, decoder_layers, dropout, cell_type, bidirectional)

print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(30, 64)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): LSTM(64, 128, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(58, 64)
    (dropout): Dropout(p=0.2, inplace=False)
    (rnn): LSTM(64, 128, num_layers=3, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=128, out_features=58, bias=True)
  )
)


In [44]:
# Load data
_, train_loader, _, _, _, _ = load_data(train_input,train_output, batch_size=64)

# Train
for epoch in range(num_epochs):
    model, train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)

    print(f"Epoch {epoch + 1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}% | ")
    

Epoch 1/10 | Train Loss: 1.3840 | Train Accuracy: 36.63% | 
Epoch 2/10 | Train Loss: 1.2593 | Train Accuracy: 37.00% | 
Epoch 3/10 | Train Loss: 1.2022 | Train Accuracy: 37.16% | 
Epoch 4/10 | Train Loss: 1.1785 | Train Accuracy: 37.26% | 
Epoch 5/10 | Train Loss: 1.1493 | Train Accuracy: 37.29% | 
Epoch 6/10 | Train Loss: 1.1214 | Train Accuracy: 37.29% | 
Epoch 7/10 | Train Loss: 1.0948 | Train Accuracy: 37.30% | 
Epoch 8/10 | Train Loss: 1.0676 | Train Accuracy: 37.37% | 
Epoch 9/10 | Train Loss: 1.0419 | Train Accuracy: 37.37% | 
Epoch 10/10 | Train Loss: 1.0208 | Train Accuracy: 37.37% | 


In [45]:
# Load data
_, test_loader, _, _, _, _ = load_data(test_input,test_output, batch_size=64)
test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
print(f"Test Accuracy: {test_accuracy:.2f}% | ")
 

Test Accuracy: 30.13% | 


In [27]:
def decode_prediction_indices(predicted_indices, index_to_token, target_vocab):
    """
    Converts a list of predicted token indices into a decoded string,
    excluding special tokens like <pad>, <sos>, and <eos>.
    """
    filtered_indices = []
    for idx in predicted_indices:
        if idx in index_to_token and idx not in (target_vocab['<pad>'], target_vocab['<sos>'], target_vocab['<eos>']):
            filtered_indices.append(idx)
    
    decoded_string = ''
    for idx in filtered_indices:
        decoded_string += index_to_token[idx]
    
    return decoded_string


def decode_target_indices(target_indices, index_to_token, target_vocab):
    """
    Converts target indices to a decoded string, adjusting indices if needed.
    Handles specific logic to shift indices >= 10 by -3.
    """
    filtered_indices = []
    for idx in target_indices:
        if idx in index_to_token and idx not in (target_vocab['<pad>'], target_vocab['<sos>'], target_vocab['<eos>']):
            corrected_idx = idx if idx < 10 else idx - 3
            filtered_indices.append(corrected_idx)
    
    decoded_string = ''
    for idx in filtered_indices:
        decoded_string += index_to_token[idx]
    
    return decoded_string


def predict_sequences(model, dataloader, device):
    """
    Uses the trained model to predict output sequences from input batches.
    Returns:
    - A list of tuples containing (input_sequence, predicted_output)
    - A list of ground truth target sequences
    """
    model.eval()
    predictions = []
    ground_truths = []

    with torch.no_grad():
        for source_batch, target_batch in dataloader:
            source_batch = source_batch.to(device)
            target_batch = target_batch.to(device)

            # Get model output with no teacher forcing
            output_logits = model(source_batch, target_batch, teacher_forcing_ratio=0.0)
            predicted_tokens = output_logits.argmax(dim=2)  # (batch_size, target_len)

            # Move tensors to CPU for further processing
            predictions.append((source_batch.cpu().numpy(), predicted_tokens.cpu().numpy()))
            ground_truths.append(target_batch.cpu().numpy())

    return predictions, ground_truths


In [28]:
dataset, dataloader, input_vocab, target_vocab, max_input_len, max_target_len = load_data(test_input, test_output,batch_size = 64)
# Make sure to define the reverse dictionaries for converting indices back to text
latin_idx2token = {idx: char for char, idx in input_vocab.items()}
urdu_idx2token = {idx: char for char, idx in target_vocab.items()}

In [34]:
# Run predictions on the Urdu test set using the trained model
predicted_batches, ground_truth_batches = predict_sequences(model, test_loader, device)

# This will store final results for each sequence: [input_text, ground_truth_text, predicted_text]
sequence_translation_results = []

# Iterate over batches of predictions and corresponding ground truth
for (batch_input_indices, batch_predicted_indices), batch_actual_indices in zip(predicted_batches, ground_truth_batches):

    # Iterate through each sample in the batch
    for sample_idx in range(batch_input_indices.shape[0]):
        # Decode the Latin-script input indices into a readable string
        input_text = decode_prediction_indices(
            batch_input_indices[sample_idx],
            latin_idx2token,
            input_vocab
        )

        # Decode the actual Urdu target indices into a readable string
        actual_text = decode_prediction_indices(
            batch_actual_indices[sample_idx],
            urdu_idx2token,
            target_vocab
        )

        # Decode the predicted Urdu output indices into a readable string (handling index shift if needed)
        predicted_text = decode_target_indices(
            batch_predicted_indices[sample_idx],
            urdu_idx2token,
            target_vocab
        )

        # Save the result for later inspection or logging
        sequence_translation_results.append([input_text, actual_text, predicted_text])


In [35]:
with open('predictions_vanilla.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Input Text', 'Actual Target', 'Predicted Text'])
    writer.writerows(sequence_translation_results)

In [36]:
df1 = pd.read_csv('predictions_vanilla.csv')
df1

,Input Text,Actual Target,Predicted Text
0,aaieen,آئین,ںہك
1,aaoin,آئین,ںمئہ
2,aayean,آئین,ںہئہک
3,aayeen,آئین,ںہئہک
4,aayin,آئین,ںہئہ
...,...,...,...
10512,yegangat,یگانگت,ںكجا
10513,ehan,یہان,ںاك
10514,yahaan,یہان,ںہاك
10515,yahan,یہان,ںہاک
